In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import time

In [2]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
service = Service(executable_path=ChromeDriverManager().install())

[WDM] - Downloading:  82%|███████████████████████████████████████████████████████████████████▏              | 5.16M/6.29M [00:00<00:00, 12.3MB/s]

In [43]:
def get_comments(service, df, title_no, episode_no):
    TIMEOUT = 1   
    driver = webdriver.Chrome(service=service)
    
    # https://www.webtoons.com/en/action/omniscient-reader/episode-109/viewer?title_no=2154&episode_no=110
    url = 'https://www.webtoons.com/en/action/omniscient-reader/episode-109/viewer?'
    episode_url = url + 'title_no=' + str(title_no) + '&episode_no='+str(episode_no)
    print(episode_url)
    driver.get(episode_url)
    driver.implicitly_wait(TIMEOUT)
    
    # No iframe
   
    # No Cleanbot filtering setting

    # No need to click "전체 댓글"
    
    # Move to the last page and Get the total number of pages
    driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > a.u_cbox_next.u_cbox_next_end').click()
    time.sleep(1)
    strong = driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > strong')
    num_pages = strong.get_attribute('data-param')
    print("num_pages:", num_pages)

    # Move to the first page
    driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > a.u_cbox_pre.u_cbox_pre_end').click()
    time.sleep(1)
        
    # 0 ~ 446 
    for page in range(int(num_pages)):

        soup = bs(driver.page_source,"html.parser")

        # Extract dates
        dates = soup.findAll("span", {"class":["u_cbox_date"]})
        dates = [date.text for date in dates]        

        # Extract reveiws
        reviews = soup.findAll("span", {"class":["u_cbox_contents"]})
        reviews = [review.get_text() for review in reviews]

        df_page = pd.DataFrame(data=[], columns=['Episode','Date','Review'])       
        df_page['Date'] = dates
        df_page['Review'] = reviews
        df_page['Episode'] = df_page['Episode'].fillna(int(episode_no))
        
        df = pd.concat([df,df_page])      
        print(page+1, " page Done.")        

        if (page+1) == int(num_pages):
            return df
        
        # Move to the next page
        driver.find_element(By.CSS_SELECTOR, "#cbox_module > div > div.u_cbox_paginate > div > strong + a").click()        
        time.sleep(0.4)
    return df

df = pd.DataFrame(data=[], columns=['Episode','Date','Review'])

# omniscient-reader - Episode 1
title_no = 2154
episode_no = 1
df = get_comments(service, df, title_no, episode_no)

# Save dataframe
DATA_PATH = './data/'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
file_path = './data/omniscient-reader_Episode_{episode_no}.csv'.format(episode_no=episode_no)
df.to_csv(file_path,index = False)
print("Save Done.")
df

https://www.webtoons.com/en/action/omniscient-reader/episode-109/viewer?title_no=2154&episode_no=1
num_pages: 447
1  page Done.
2  page Done.
3  page Done.
4  page Done.
5  page Done.
6  page Done.
7  page Done.
8  page Done.
9  page Done.
10  page Done.
11  page Done.
12  page Done.
13  page Done.
14  page Done.
15  page Done.
16  page Done.
17  page Done.
18  page Done.
19  page Done.
20  page Done.
21  page Done.
22  page Done.
23  page Done.
24  page Done.
25  page Done.
26  page Done.
27  page Done.
28  page Done.
29  page Done.
30  page Done.
31  page Done.
32  page Done.
33  page Done.
34  page Done.
35  page Done.
36  page Done.
37  page Done.
38  page Done.
39  page Done.
40  page Done.
41  page Done.
42  page Done.
43  page Done.
44  page Done.
45  page Done.
46  page Done.
47  page Done.
48  page Done.
49  page Done.
50  page Done.
51  page Done.
52  page Done.
53  page Done.
54  page Done.
55  page Done.
56  page Done.
57  page Done.
58  page Done.
59  page Done.
60  page D

,Episode,Date,Review
0,1,"Aug 19, 2020",This art is HONESTLY SPECTACULAR. Can't WAIT t...
1,1,"Aug 19, 2020",IT'S HERE!!!
2,1,"Aug 19, 2020","If he read the story over ten years, I hope he..."
3,1,"Aug 19, 2020",Yo this better become the next big action fant...
4,1,"Aug 19, 2020","DONT SLEEP ON THIS, THIS IS A TOP TIER MANHWA"
...,...,...,...
14,1,"Sep 1, 2021","wait, sung jin woo????"
0,1,"Sep 3, 2020",Off brand ass solo leveling
1,1,"Jul 12, 2021",is this solo leveling?
2,1,"Aug 27, 2020",just a warning: prepare yourself for blood spl...


In [44]:
df

,Episode,Date,Review
0,1,"Aug 19, 2020",This art is HONESTLY SPECTACULAR. Can't WAIT t...
1,1,"Aug 19, 2020",IT'S HERE!!!
2,1,"Aug 19, 2020","If he read the story over ten years, I hope he..."
3,1,"Aug 19, 2020",Yo this better become the next big action fant...
4,1,"Aug 19, 2020","DONT SLEEP ON THIS, THIS IS A TOP TIER MANHWA"
...,...,...,...
14,1,"Sep 1, 2021","wait, sung jin woo????"
0,1,"Sep 3, 2020",Off brand ass solo leveling
1,1,"Jul 12, 2021",is this solo leveling?
2,1,"Aug 27, 2020",just a warning: prepare yourself for blood spl...
